<a href="https://colab.research.google.com/github/sherlockieee/cs152/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [116]:
NAME = "Ha Tran Nguyen Phuong"
COLLABORATORS = ""

---

<h1>CS152 Assignment 3: The DPLL Algorithm</h1>

<h1>Question 1</h1>

Define your <code>Literal</code> class below.  Ensure that you define a <code>name</code> and <code>sign</code> attribute as required in the assignment description.  In addition, include any other attributes and member functions as needed. You will need to overload <code>__neg__()</code> to correctly handle negated literals.  You may need to overload <code>__eq__()</code> and <code>__hash__()</code> also.

In [117]:
# Import any packages you need here
# Also define any variables as needed


class Literal:
    def __init__(self, name, sign = True):
        self.name = name
        self.sign = sign
    
    def __neg__(self):
        return Literal(self.name, sign = not self.sign)

    def __repr__(self):
        if not self.sign:
            return f"-{self.name}"
        return f"{self.name}"
    
    def __hash__(self):
        return hash(self.name)
    
    def __eq__(self, other):
        return self.name == other.name

    def __lt__(self, other):
        """Less than operator to implement alphabetical order"""
        return self.name > other.name
    
    def __gt__(self, other):
        """Greater than operator to implement alphabetical order"""
        return self.name < other.name



<h1>Question 2</h1>
Implement DPLL by filling in the stubs below.  Note that the various heuristics are not required to be implemented for basic functionality, but a template has been provided for you to do so if you attempt the extension questions

In [118]:
# Define degree, pure symbol and unit clause heuristics here (optional).
# Add in your functions under the templates below
# YOUR CODE HERE (OPTIONAL)

from collections import defaultdict

def DPLL_Satisfiable(KB, heuristic_level=0):
    '''Takes in a KB and returns whether the KB is satisfiable, the model that makes it so
    and the symbol list that is explored
    (satisfiable, sat_model, symbol_list) = DPLL_Satisfiable(KB, heuristic_level)
        
    Parameters:
    -----------
    KB: list[Set(Literal)]
        A knowledge base of clauses (CNF) consisting of a list of sets of literals.  
        A KB might look like [{A,B},{-A,C,D}]
    heuristic_level: int
        An integer that will be passed in to specify which heuristics to implement.
        heuristic_level = 0: No heuristic
        heuristic_level = 1: Degree heuristic
        heuristic_level = 2: Unit clause and pure symbol heuristic
        heuristic_level = 3: Unit clause, pure symbol and degree heuristic
    
    Returns:
    -----------
    Tuple(bool, dict{Literal: bool}, list[Literal])
        satisfiable: Returns True if the KB is satisfiable, or False otherwise
        sat_model: A dictionary that assigns a truth value to each literal for the model 
        that satisfies KB. For example, a model might look like {A: True, B: False}
        symbol_list: list of literal objects that indicate the order in which literals are assigned
    '''
    #Get all symbols
    symbols = set()
    for clause in KB:
        for literal in clause:
            symbols.add(literal if literal.sign else -literal)
    return DPLL(KB, symbols, {}, heuristic_level)

def DPLL(clauses, symbols, model, heuristic_level, symbol_list = None):
    '''Takes in a list of clauses and returns whether the KB is satisfiable, 
     the model that makes it so, and the symbol list that is explored
    (satisfiable, sat_model, symbol_list) = DPLL(clauses, symbols, model, heuristic_level, symbol_list)
        
    Parameters:
    -----------
    clauses: list[Set(Literal)]
        A knowledge base of clauses (CNF) consisting of a list of sets of literals.  
        A KB might look like [{A,B},{-A,C,D}]
    symbols: set(Literal)
        A set of all symbols used in the clauses that have not been explored
    model: dict{Literal: bool}
        The model created thus far, consisting of the literal and its associated value
    heuristic_level: int
        An integer that will be passed in to specify which heuristics to implement.
        heuristic_level = 0: No heuristic
        heuristic_level = 1: Degree heuristic
        heuristic_level = 2: Unit clause and pure symbol heuristic
        heuristic_level = 3: Unit clause, pure symbol and degree heuristic
    symbol_list: list/ None
        A list containing which symbols have been explored so far
    
    Returns:
    -----------
    Tuple(bool, dict{Literal: bool}, list[Literal])
        satisfiable: Returns True if the KB is satisfiable, or False otherwise
        sat_model: A dictionary that assigns a truth value to each literal for the model 
        that satisfies KB. For example, a model might look like {A: True, B: False}
        symbol_list: list of literal objects that indicate the order in which literals are assigned
    '''

    #if every clause is true, we would get an empty list of clauses
    #then we don't need to evaluate the rest and can just return the model
    if not clauses:
        return True, model, symbol_list

    #check if any clause is all false in the model
    if set() in clauses:
        return False, {}, []

    #initialize list to avoid Python's mutable problem
    if symbol_list is None:
        symbol_list = []
    
    def degree_heuristic(clauses):
        '''Find the symbol that occur over the most clauses
        
        Parameters:
        -----------
        clauses: list[Set(Literal)]
            A knowledge base of clauses (CNF) consisting of a list of sets of literals.  
            A KB might look like [{A,B},{-A,C,D}]
        
        Returns:
        -----------
        Literal
            The literal with the highest degree'''
        counts = defaultdict(int)
        for clause in clauses:
            for symbol in clause:
                #only take positive symbol
                abs_symbol = symbol if symbol.sign else -symbol
                counts[abs_symbol] += 1
        P = max([(count, symbol) for symbol, count in counts.items()])[1]
        return P

    def find_pure_symbol(clauses):
        '''Inner function, find all pure symbols in the remaining clauses
        
        Parameters:
        -----------
        clauses: list[Set(Literal)]
            A knowledge base of clauses (CNF) consisting of a list of sets of literals.  
            A KB might look like [{A,B},{-A,C,D}]
        
        Returns:
        -----------
        list[Literal]
            A list of pure symbols'''
        pure = {}
        for clause in clauses:
            for symbol in clause:
                if symbol not in pure:
                    pure[symbol] = symbol.sign
                elif pure[symbol] != symbol.sign:
                    pure[symbol] = None
        return [symbol for symbol in pure.keys() if pure[symbol] is not None]

    def unit_clause_heuristic(clauses):
        '''Inner function, find all literals satisfying the unit clause heuristic
        
        Parameters:
        -----------
        clauses: list[Set(Literal)]
            A knowledge base of clauses (CNF) consisting of a list of sets of literals.  
            A KB might look like [{A,B},{-A,C,D}]
        
        Returns:
        -----------
        list[Literal]
            A list of symbols satisfying the unit clause heuristic'''
        return [symbol for clause in clauses for symbol in clause if len(clause) == 1]

    #if heuristic level = 0, just get a random P 
    if heuristic_level == 0:
        #get the first element in set
        for P in symbols:
            break

    #apply degree heuristic if heuristic level = 1
    if heuristic_level == 1:
        P = degree_heuristic(clauses)

    #apply unit clause and pure symbol heuristic if heuristic level >= 2
    if heuristic_level >= 2:
        all_pure = find_pure_symbol(clauses)
        all_unit_clause = unit_clause_heuristic(clauses)
        #find all symbols that are either pure or part of unit clauses
        #and are in the remaining list of symbols
        all_heuristic = (set(all_pure) | set(all_unit_clause)) & symbols
        if not all_heuristic:
            P = degree_heuristic(clauses)

        #choose P alphabetically if heuristic level = 2
        elif heuristic_level == 2:
            P = max(all_heuristic)

        #use degree heuristic among clauses that contain the symbols if heuristic level = 3
        elif heuristic_level == 3:
            new_clauses = []
            for clause in clauses:
                for symbol in clause:
                    if symbol in all_heuristic:
                        new_clauses.append(clause)
                        break

            P = degree_heuristic(new_clauses)


    rest = symbols - {P}

    def create_clause_and_model(clauses, model, P, sign):
        '''Create new clauses and model for a new P with a particular sign
        
        Parameters:
        -----------
        clauses: list[Set(Literal)]
            A knowledge base of clauses (CNF) consisting of a list of sets of literals.  
            A KB might look like [{A,B},{-A,C,D}]
        model: dict{Literal: bool}
            The model created so far, consisting of the literal and its associated value
        P: Literal
            The literal to be removed from the clauses
        sign: bool
            The sign of the particular literal
        
        Returns:
        -----------
        Tuple(list[Set(Literal)], dict{Literal: bool})
            A tuple consisting of two values
            The first is the new clauses after we assign P a particular sign
            The second is the new model with P inside'''
        new_clauses = []

        for clause in clauses:
            #if P is not in the old clause, there's no change to be made
            if P not in clause:
                new_clause = set(clause)
                new_clauses.append(new_clause)
            else:
                #if P is in the clause
                for sym in clause:
                    if sym == P:
                        #if it's of a different sign
                        #we remove this from the set so it cannot be evaluated
                        if sym.sign != sign:
                            new_clause = set(clause - {P})
                            new_clauses.append(new_clause)
                        break
        #create a new model with P
        new_model = {P: sign}
        new_model.update(model)
        return new_clauses, new_model

    #do recursive function when P is True
    positive_clauses, positive_model = create_clause_and_model(clauses, model, P, True)
    positive = DPLL(positive_clauses, rest, positive_model, heuristic_level, symbol_list + [P])
    #if we find a result, we simple need to return the found model
    if positive[0]:
        return positive
    #do recursive function when P is False
    negative_clauses, negative_model = create_clause_and_model(clauses, model, P, False)
    negative = DPLL(negative_clauses, rest, negative_model, heuristic_level, symbol_list + [P])
    #if we find a result, we simple need to return the found model
    if negative[0]:
        return negative
    #if P can't be both negative and positive, the model cannot be satisfied
    return False, {}, []

DPLL_Satisfiable(KB,3)




(True, {E: False, D: True, B: False, A: False}, [D, B, E, A])

<h1>Question 3</h1>

Implement your KB from Russell & Norvig in CNF by filling in the sets inside the list <code>KB</code> below.  Ensure that your KB is in a list of set format as stated in the assignment instructions.

$ S1: A \iff (B \lor E)$

$ \Rightarrow (A \Rightarrow (B \lor E) ) \land ((B \lor E) \Rightarrow A )$


$ \Rightarrow (\neg A \lor B \lor E ) \land (\neg (B \lor E) \lor A )$

$ \Rightarrow (\neg A \lor B \lor E ) \land ((\neg B \land \neg E) \lor A )$

$ \Rightarrow (\neg A \lor B \lor E ) \land (\neg B \lor A) \land (\neg E \lor A )$

$ S2: E \Rightarrow D $

$ \Rightarrow \neg E \lor D $


$ S3: C \land F \Rightarrow \neg B $

$ \Rightarrow \neg (E \land F) \lor \neg B $

$ \Rightarrow \neg E \lor \neg F \lor \neg B $

$ S4: E \Rightarrow B $

$ \Rightarrow \neg E \lor B $

$ S5: B \Rightarrow F $

$ \Rightarrow \neg B \lor F $

$ S6: B \Rightarrow C $

$ \Rightarrow \neg B \lor C $

In [119]:
# Define some literals
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

# YOUR CODE HERE
KB = [{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}]

DPLL_Satisfiable(KB, 3)

(True, {E: False, D: True, B: False, A: False}, [D, B, E, A])

<h1>Extensions</h1>

1. Implement the degree heuristic for choosing symbols.  If <code>heuristic_level=1</code>, then the degree heuristic should be used to select which symbol to assign.
2. Implement the pure symbol and unit clause heuristics.  If <code>heuristic_level=2</code>, then these heuristics should be used to find select first a pure symbol, and if one is not found, then a unit clause.  If neither pure symbols nor unit clauses are found, then the degree heuristic should be used to select a symbol.  If there are multiple pure symbols or unit clauses found, then use alphabetical order to select amongst them.
3. Modify the pure symbol heuristic to choose the pure symbol that occurs in the most number of clauses.  This should be activated if <code>heuristic_level=3</code>.

<h1>Basic Functionality Tests</h1>

All of the tests in this section must be passed for the code to be considered fully functional.  Other unseen test cases will be used after submission to further verify functionality

In [120]:
# This section will test the correct definition of the literal class


# Test the name attribute works correctly
assert(A.name == 'A')

# Test that negation works correctly
assert(not (-C).sign)

In [121]:
# This section will test that the KB has been correctly converted to CNF by performing 
# tests over all possible models
import itertools
symbols = {A,B,C,D,E,F}
symbol_list = [A,B,C,D,E,F]

def evaluate_russell_norvig_KB(model):
    # Manually evaluate the KB using the model
    s = list()
    s.append(model[A] == (model[B] or model[E]))
    s.append(model[E] <= model[D])
    s.append((model[C] and model[F]) <= (not model[B]))
    s.append(model[E] <= model[B])
    s.append(model[B] <= model[F])
    s.append(model[B] <= model[C])
    return all(s)

def evaluate_KB(KB, model):
    model_true = True
    model_name_dict = {l.name: model[l] for l in model}
    for clause in KB:
        evaluation = {l.sign == model_name_dict[l.name] for l in clause if l.name in model_name_dict}
        model_true = model_true and any(evaluation)
    return model_true

all_models = list(itertools.product([False, True], repeat=6))
for i in range(0, len(all_models)):
    # Test all truth values
    test_model = {symbol_list[j]: all_models[i][j] for j in range(0,6)}
    
    # Test the model
    assert(evaluate_russell_norvig_KB(test_model) == evaluate_KB(KB, test_model))

In [122]:
# This section will test the basic working of DPLL
# Satisfiable model
# test_KB = [{A, C},{-A, C}, {B, -C}]
# t, model, symbol_trace = DPLL_Satisfiable(test_KB)
# assert(evaluate_KB(test_KB, model))
# assert(t)

# Unsatisfiable model
test_KB = [{A, C},{-A, C}, {-C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
print(t, model, symbol_trace)
assert(not t)

False {} []


In [123]:
# This will test DPLL on the KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,0)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model)) 

<h1>Extension Tests</h1>

This section will test that the degree heuristic, pure symbol and unit clause heuristics are correctly implemented.  Note that in your code, the <code>heuristic_level</code> needs to correctly activate the heuristic being tested, and the <code>symbol_trace</code> needs to be correctly generated 

In [124]:
# Degree Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,1)
print(model)
print(symbol_trace)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))  

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['B','F', 'E', 'C', 'C', 'E', 'A'], 
                                          ['B', 'A', 'C', 'F', 'E', 'A'], 
                                          ['B','E','A']])

symbol_trace

{A: False, E: False, B: False}
[B, E, A]


[B, E, A]

In [125]:
# Pure Symbol & Unit Clause Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,2)
print(model)
print(symbol_trace)


# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['D', 'B', 'E', 'A', 'C', 'F', 'E', 'A'], 
                                          ['D', 'B', 'A', 'C', 'F', 'E', 'A'], 
                                          ['D', 'B' ,'E', 'A']])

{-A: False, -E: False, B: False, D: True}
[D, B, -E, -A]


In [126]:
# Pure Symbol & Unit Clause Heuristic Test, choosing the most-frequently used pure symbol, for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,3)
print(model)
print(symbol_trace)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['D', 'B', 'A', 'C', 'F', 'E', 'A'], 
                                          ['D', 'B' ,'E', 'A']])

{A: False, E: False, B: False, D: True}
[D, B, E, A]
